# Sumário

1. [Pré-Processamento](Pré)
2. [Machine Learning](ml)
3. [Gráficos](gf)



# Pré-Processamento dados Reais


In [2]:
import pandas as pd
import os
import numpy as np


In [3]:

df1 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/df1.csv')# falha de subcarga
df2 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/df2.csv')# sem falha de subcarga


In [4]:
df10 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/df10.csv') # falha de subcarga 10%
df20 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/df20.csv') # falha de subcarga 20%
df15 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/df15.csv') # falha de subcarga 15%
dfbs = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/dfbs.csv') #  sem de falha de subcarga

In [5]:
import pandas as pd


# Obter os nomes das colunas de ambos os DataFrames
columns_df2 = set(df2.columns)
columns_dfbs = set(dfbs.columns)

# Encontrar as colunas com nomes diferentes
different_columns_df2 = columns_df2.difference(columns_dfbs)
different_columns_dfbs = columns_dfbs.difference(columns_df2)

# Criar listas vazias para as colunas do DataFrame
different_columns_df2_list = list(different_columns_df2)
different_columns_dfbs_list = list(different_columns_dfbs)

# Preencher com None para manter as listas com o mesmo tamanho
max_length = max(len(different_columns_df2_list), len(different_columns_dfbs_list))
different_columns_df2_list.extend([None] * (max_length - len(different_columns_df2_list)))
different_columns_dfbs_list.extend([None] * (max_length - len(different_columns_dfbs_list)))

# Criar um DataFrame para a tabela
table_data = {
    'Colunas do df2 com nomes diferentes do dfbs': different_columns_df2_list,
    'Colunas do dfbs com nomes diferentes do df2': different_columns_dfbs_list
}

result_table = pd.DataFrame(table_data)


result_table

,Colunas do df2 com nomes diferentes do dfbs,Colunas do dfbs com nomes diferentes do df2
0,RTU_REFG_SUCT_TEMP_2,RTU_SEN_CAPA
1,RTU_REFG_DISC_PRES_1,RTU_REFG_DISC_PRES
2,time,RTU_REFG_SUCT_TEMP
3,RTU_REFG_SUCT_TEMP_1,RTU_TOT_CAPA
4,date,RTU_RA_FLOW
5,RTU_REFG_COND_TEMP_2,RTU_REFG_DISC_TEMP
6,RTU_REFG_COND_TEMP_1,RTU_STG_STA
7,RTU_REFG_SUCT_PRES_1,RTU_REFG_COND_PRES
8,RTU_REFG_SUCT_PRES_2,RTU_REFG_SUCT_PRES
9,RTU_REFG_DISC_TEMP_1,RTU_REFG_COND_TEMP


In [6]:
# Remover as features incongruentes

columns_to_drop_real = ['Unnamed: 0','time','date','month'
    'RTU_MA_HUM', 'RTU_MA_TEMP', 'RTU_LA_COND_TEMP',]

In [7]:
df1_t = df1.assign(Target = 1) # falha de subcarga

df2_t = df2.assign(Target = 0) # não falha de subcarga



In [8]:
# Eliminando  os momentos de desligameno do sistema no  site 2 de forma continua
# Seleciona as linhas que não contêm zero na FAN_WAAT e 'COMP_WATT'.

df1_c = df1_t.loc[(df1_t['RTU_SA_FAN_WATT'] != 0) & (df1_t['RTU_COMP_WATT'] != 0)]
df2_c = df2_t.loc[(df2_t['RTU_SA_FAN_WATT'] != 0) & (df2_t['RTU_COMP_WATT'] != 0)]

In [9]:
# Cria uma nova feature com a soma das pressões
df1_c = df1_c.assign(
    RTU_REFG_DISC_PRES=df1_c['RTU_REFG_DISC_PRES_1'] + df1_c['RTU_REFG_DISC_PRES_2'],
    RTU_REFG_SUCT_PRES=df1_c['RTU_REFG_SUCT_PRES_1'] + df1_c['RTU_REFG_SUCT_PRES_2'])

df2_c = df2_c.assign(
    RTU_REFG_DISC_PRES=df2_c['RTU_REFG_DISC_PRES_1'] + df2_c['RTU_REFG_DISC_PRES_2'],
    RTU_REFG_SUCT_PRES=df2_c['RTU_REFG_SUCT_PRES_1'] + df2_c['RTU_REFG_SUCT_PRES_2'])


#### Padronizar a Pressão 

In [14]:
# Passo 1: Calcular a Derivada da Pressão (N-1) por Minuto

# Certifique-se de que as colunas de data/hora estão no formato de data/hora do pandas
df1_c['Datetime'] = pd.to_datetime(df1_c['Datetime'])

df2_c['Datetime'] = pd.to_datetime(df1_c['Datetime'])

# Ordene o DataFrame pela coluna 'DataHora' (caso não esteja ordenado)
df1_c.sort_values(by='Datetime', inplace=True)

df2_c.sort_values(by='Datetime', inplace=True)


# Calcule a derivada por minuto para 'RTU_REFG_DISC_PRES'

df1_c['Derivada_RTU_REFG_DISC_PRES'] = df1_c['RTU_REFG_DISC_PRES'].diff() / df1_c['Datetime'].diff().dt.total_seconds()
df2_c['Derivada_RTU_REFG_DISC_PRES'] = df2_c['RTU_REFG_DISC_PRES'].diff() / df2_c['Datetime'].diff().dt.total_seconds()


# Calcule a derivada por minuto para 'RTU_REFG_SUCT_PRES'
df1_c['Derivada_RTU_REFG_SUCT_PRES'] = df1_c['RTU_REFG_SUCT_PRES'].diff() / df1_c['Datetime'].diff().dt.total_seconds()

df2_c['Derivada_RTU_REFG_SUCT_PRES'] = df2_c['RTU_REFG_SUCT_PRES'].diff() / df2_c['Datetime'].diff().dt.total_seconds()



In [16]:
# Passo 2 Calcular a Média móvel 

# Calcule a média móvel da derivada por minuto para 'RTU_REFG_DISC_PRES'

janela_um_dia = 1440   # Janela de um dia (1440 minutos)

df1_c['MediaMovel_Derivada_RTU_REFG_DISC_PRES'] = df1_c['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_um_dia).mean()

# Calcule a média móvel da derivada por minuto para 'RTU_REFG_SUCT_PRES'
df1_c['MediaMovel_Derivada_RTU_REFG_SUCT_PRES'] = df1_c['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_media_movel).mean()

In [ ]:
# Passo 3 Calcular o Desvio Padrão


janela_um_dia = 1440 # Defina a janela de 1 dia (1440 minutos)


# Calcule o desvio padrão da derivada por minuto dentro da janela de 1 dia para 'RTU_REFG_DISC_PRES'
df1_c['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES'] = df1_c['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_um_dia).std()

# Calcule o desvio padrão da derivada por minuto dentro da janela de 1 dia para 'RTU_REFG_SUCT_PRES'
df1_c['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES'] = df1_c['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_um_dia).std()


In [ ]:
# Passo 4 - Padronizar Manualmente

# Certifique-se de que as colunas de data/hora estão no formato de data/hora do pandas
df1_c['Datetime'] = pd.to_datetime(df1_c['Datetime'])

# Calcule o Z-Score para cada minuto usando a média e o desvio padrão das colunas
df1_c['Z_Score_Derivada_RTU_REFG_SUCT_PRES'] = (df1_c['Derivada_RTU_REFG_SUCT_PRES'] - df1_c['MediaMovel_Derivada_RTU_REFG_SUCT_PRES']) / df1_c['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES']

df1_c['Z_Score_Derivada_RTU_REFG_DISC_PRES'] = (df1_c['Derivada_RTU_REFG_DISC_PRES'] - df1_c['MediaMovel_Derivada_RTU_REFG_DISC_PRES']) / df1_c['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES']



In [ ]:
df1_c.tail()

,Unnamed: 0,Datetime,RTU_COMP_WATT,RTU_LA_COND_TEMP,RTU_MA_HUM,RTU_MA_TEMP,RTU_OA_HUM,RTU_OA_TEMP,RTU_RA_HUM,RTU_RA_TEMP,...,RTU_REFG_DISC_PRES,RTU_REFG_SUCT_PRES,Derivada_RTU_REFG_DISC_PRES,Derivada_RTU_REFG_SUCT_PRES,MediaMovel_Derivada_RTU_REFG_DISC_PRES,MediaMovel_Derivada_RTU_REFG_SUCT_PRES,DesvioPadrao_Derivada_RTU_REFG_DISC_PRES,DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES,Z_Score_Derivada_RTU_REFG_SUCT_PRES,Z_Score_Derivada_RTU_REFG_DISC_PRES
41441,41441,2020-06-29 22:40:00,367.000031,25.635582,82.265457,20.686346,92.883644,12.763875,75.750832,23.729204,...,16.245808,15.658225,-0.042304,0.046764,0.000083,-0.000128,0.020163,0.011237,4.173039,-2.102181
41442,41442,2020-06-29 22:41:00,1239.000122,25.595110,82.305046,20.695372,92.923515,12.715758,75.831161,23.729204,...,17.591895,14.597172,0.022435,-0.017684,0.000097,-0.000141,0.020172,0.011246,-1.559930,1.107383
41455,41455,2020-06-29 22:54:00,1187.000122,23.231857,81.754951,20.939980,93.978607,12.881154,75.283546,23.952757,...,18.028032,14.904505,0.000559,0.000394,0.000097,-0.000141,0.020172,0.011246,0.047533,0.022895
41459,41459,2020-06-29 22:58:00,201.000015,22.209787,81.706779,20.944384,94.160515,12.902560,75.295967,24.010971,...,18.110584,15.096486,0.000344,0.000800,0.000099,-0.000140,0.020172,0.011246,0.083550,0.012167
41460,41460,2020-06-29 22:59:00,940.000061,24.340851,81.684425,20.950773,94.150856,12.874650,75.238152,23.994673,...,16.361328,16.031805,-0.029154,0.015589,0.000077,-0.000129,0.020187,0.011254,1.396628,-1.448064


In [ ]:
# Elimina as features
columns_to_drop = ['Unnamed: 0',
    'RTU_MA_HUM', 'RTU_MA_TEMP', 'RTU_LA_COND_TEMP',
    'RTU_REFG_DISC_PRES_1', 'RTU_REFG_DISC_PRES_2',
    'RTU_REFG_SUCT_PRES_1', 'RTU_REFG_SUCT_PRES_2',
    'RTU_REFG_COND_TEMP_1', 'RTU_REFG_COND_TEMP_2',
    'RTU_REFG_SUCT_TEMP_1', 'RTU_REFG_SUCT_TEMP_2',
    'RTU_REFG_DISC_TEMP_1', 'RTU_REFG_DISC_TEMP_2',
]

df1_x = df1_c.drop(columns_to_drop, axis=1)
df2_x = df2_c.drop(columns_to_drop, axis=1)

In [ ]:
# Converter fahrenheit para celsius
def fahrenheit_to_celsius(fahrenheit):
    celsius = (fahrenheit - 32) * 5/9
    return celsius

# Colunas a serem convertidas
columns_to_convert = ['RTU_RA_TEMP', 'RTU_OA_TEMP', 'ZA_TEMP', 'RTU_REFG_COND_TEMP',
                      'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP','RTU_SA_TEMP']


# Aplicando a conversão para Celsius
df20[columns_to_convert] = df20[columns_to_convert].apply(fahrenheit_to_celsius)
dfbs[columns_to_convert] = dfbs[columns_to_convert].apply(fahrenheit_to_celsius)

In [ ]:
# Converter colunas de Pascal para Bar
df20['RTU_REFG_SUCT_PRES'], df20['RTU_REFG_DISC_PRES'] = df20['RTU_REFG_SUCT_PRES'] /100000 , df20['RTU_REFG_DISC_PRES'] /100000
dfbs['RTU_REFG_SUCT_PRES'], dfbs['RTU_REFG_DISC_PRES'] = dfbs['RTU_REFG_SUCT_PRES'] / 100000, dfbs['RTU_REFG_DISC_PRES'] /100000

In [ ]:

df20_t = df20.assign(Target = 1) # falha de subcarga

dfbs_t = dfbs.assign(Target = 0) #  não falha

In [ ]:
# Eliminando  os momentos de desligameno do sistema no  site 2 de forma continua
# Seleciona as linhas que não contêm zero na FAN_WAAT e 'COMP_WATT'.

df20_c = df20_t.loc[(df20_t['RTU_COMP_WATT'] != 0) ]
dfbs_c = dfbs_t.loc[(dfbs_t['RTU_COMP_WATT'] != 0) ]

In [ ]:
columns_to_drop = ['Unnamed: 0',
    'RTU_RA_FLOW', 'RTU_OA_FLOW', 'RTU_STG_STA',
    'RTU_REFG_COND_PRES', 'RTU_SEN_CAPA',
    'RTU_TOT_CAPA', 'ZA_TEMP_SPT',
]

df20_c = df20_c.drop(columns_to_drop, axis=1)
dfbs_c = dfbs_c.drop(columns_to_drop, axis=1)


In [ ]:
# Definir y real

df1_y= df1_c['Target']
df2_y= df2_c['Target']

In [ ]:
# # Definir X real
# df1_x= df1_x.drop(['Datetime','Target', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP'], axis=1)
# df2_x= df2_x.drop(['Datetime','Target', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP'], axis=1)


In [ ]:
# Definir X real sem features temp modificadas
df1_x= df1_x.drop(['Datetime','Target','RTU_RA_TEMP','RTU_OA_TEMP','ZA_TEMP', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP','RTU_SA_TEMP'], axis=1)
df2_x= df2_x.drop(['Datetime','Target','RTU_RA_TEMP','RTU_OA_TEMP','ZA_TEMP', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP','RTU_SA_TEMP'], axis=1)

In [ ]:
# Definir X sim

# df20_x= df20_c.drop(['Datetime','Target', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP'], axis=1)
# dfbs_x= dfbs_c.drop(['Datetime','Target', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP'], axis=1)

In [ ]:
# Definir X sim sem features temp modificadas

df20_x= df20_c.drop(['Datetime','Target','RTU_RA_TEMP','RTU_OA_TEMP','ZA_TEMP', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP','RTU_SA_TEMP'], axis=1)
dfbs_x= dfbs_c.drop(['Datetime','Target','RTU_RA_TEMP','RTU_OA_TEMP','ZA_TEMP', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP','RTU_SA_TEMP'], axis=1)

In [ ]:
# Definir y sim

df20_y = df20_c['Target']
dfbs_y = dfbs_c['Target']
